In [1]:
from google.colab import drive

drive.mount("./drive")

Mounted at ./drive


In [2]:
!mkdir ".tgz"
!mkdir data

In [3]:
!cp -R ./drive/MyDrive/projects/language-translation/data/* ./.tgz

In [6]:
import os, tarfile, random
files = os.listdir(".tgz")

for fname in files:
  FOLDER_NAME = fname.split(".")[0]
  FILE_PATH = os.path.join(".tgz", fname)
  tar_file = tarfile.open(FILE_PATH, "r")
  for item in tar_file:
    tar_file.extract(item, os.path.join("data", FOLDER_NAME))
  os.remove(FILE_PATH)

In [46]:
DATA_PATH = "data"
data_dict = {}
data_dict["text"] = []
data_dict["label"] = []

for item in os.listdir(DATA_PATH):
  language = item[:2]
  filename = "europarl-v7.{}.{}".format(item, language)
  FILE_PATH = os.path.join(DATA_PATH, item, filename)
  lines = None
  with open(FILE_PATH, "r", encoding="latin1") as f: lines = f.readlines()
  lines = random.sample(lines, 10000)
  lines = list(map(lambda x: x.strip(), lines))

  for line in lines:
    data_dict["text"].append(line)
    data_dict["label"].append(language)

In [47]:
import pandas as pd
import numpy as np

df = pd.DataFrame(data_dict)
df.head()

,text,label
0,Ir vÄl Vidaus rinkos ir vartotojÅ³ apsaugos k...,lt
1,"Kartu nustatomas reikalavimas, kad visi laivai...",lt
2,"Å i direktyva - dar vienas pavyzdys, kaip Euro...",lt
3,Dabar Taryba negaiÅ¡dama laiko turÄtÅ³ pradÄ...,lt
4,Å iuo atÅ¾vilgiu taip pat paminÄjau migracijo...,lt


In [49]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB

X = df.text.values
y = df.label.values

count_vec = CountVectorizer()
X = count_vec.fit_transform(X)

In [51]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.1, random_state=42)

model = MultinomialNB()
model.fit(X_train, y_train)

MultinomialNB()

In [54]:
model.score(X_train, y_train), model.score(X_test, y_test)

(0.9931833333333333, 0.9914)